Train the initial model to test the loss function based attack on the mnist dataset

In [14]:
import tensorflow as tf
import numpy as np
import os
from matplotlib import pyplot as plt
from tensorflow.python.keras import backend as K
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
# Set up the tensorflow session as same as the keras session
K.set_session(sess)

In [3]:
# Design the network architecture using Keras
# conv + maxpool + conv + maxpool + dense + softmax
from tensorflow.python.keras.layers import (Input, Dense, Conv2D, 
MaxPooling2D, Flatten)
from tensorflow.python.keras.models import Model

inputs = Input(shape=(28, 28, 1))
x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
outputs = Dense(10, activation='softmax')(x)

model = Model(inputs, outputs)
model.compile(optimizer=tf.train.AdamOptimizer(0.001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 64)          0         
__________

In [8]:
# Load the mnist dataset
(train_images, train_labels), (test_images, test_labels) = (
    tf.keras.datasets.mnist.load_data()
)

In [23]:
def create_subset(images, labels):
    new_images = []
    new_labels = []
    for image, label in zip(images, labels):
        if label != 9:
            new_images.append(image)
            new_labels.append(label)
    new_images = np.array(new_images)
    new_labels = np.array(new_labels)
    return new_images, new_labels
train_images, train_labels = create_subset(train_images, train_labels)
test_images, test_labels = create_subset(test_images, test_labels)

In [25]:
# Normalize the pixel values
train_images = train_images.reshape((-1, 28, 28, 1))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((-1, 28, 28, 1))
test_images = test_images.astype('float32') / 255

# Prepare the labels
train_labels = tf.keras.utils.to_categorical(train_labels)
test_labels = tf.keras.utils.to_categorical(test_labels)

In [26]:
def create_weightVar(shape):
    return tf.get_variable(
        "weights", shape, initializer = tf.glorot_normal_initializer())

def create_biasVar(shape):
    return tf.get_variable(
        "biases", shape, initializer = tf.zeros_initializer())

In [27]:
# Design the network architecture
# conv + maxpool + conv + maxpool + Dense + Softmax
from tensorflow.python.keras.layers import MaxPooling2D, Flatten, Dropout
from tensorflow.python.keras.models import Model

inputs = tf.placeholder(tf.float32, [None, 28,28,1])
labels = tf.placeholder(tf.float32, [None, 9])

# First convolutional layer
with tf.variable_scope("conv1"):
    # kernel shape = (kernelDim1, kernelDim2, kernelDepth, numOfKernels)
    Wconv1 =  create_weightVar((3, 3, 1, 32))
    biasConv1 = create_biasVar((32,))
    x = tf.nn.conv2d(inputs, Wconv1, strides=[1,1,1,1], padding="SAME"
                    ) + biasConv1
    x = tf.nn.relu(x)
    
x = MaxPooling2D((2, 2))(x)

# Second convolutional layer
with tf.variable_scope("conv2"):
    Wconv2 =  create_weightVar((3, 3, 32, 64))
    biasConv2 = create_biasVar((64,))
    x = tf.nn.conv2d(x, Wconv2, strides=[1,1,1,1], padding="SAME"
                    ) + biasConv2
    x = tf.nn.relu(x)

x = MaxPooling2D((2, 2))(x)

# Third convolutional layer
with tf.variable_scope("conv3"):
    Wconv3 =  create_weightVar((3, 3, 64, 64))
    biasConv3 = create_biasVar((64,))
    x = tf.nn.conv2d(x, Wconv3, strides=[1,1,1,1], padding="SAME"
                    ) + biasConv3
    x = tf.nn.relu(x)

x = MaxPooling2D((2, 2))(x)

x = Flatten()(x)

# Dense layer
with tf.variable_scope("FC"):
    Wdense = create_weightVar((576, 64))
    biasDense = create_biasVar((64,))
x = tf.nn.relu(tf.matmul(x, Wdense) + biasDense)

# Output layer
with tf.variable_scope("out"):
    Wout = create_weightVar((64, 9))
    biasOut = create_biasVar((9,))

logits = tf.matmul(x, Wout) + biasOut
outputs = tf.nn.softmax(logits)

# Measure accuracy
from tensorflow.python.keras.metrics import (
    categorical_accuracy as accuracy)
acc_value = tf.reduce_mean(accuracy(labels, outputs))

# Model Prediction
predicted_class = tf.argmax(outputs, axis=1)

In [28]:
# Define cross_entropy loss
from tensorflow.python.keras.losses import categorical_crossentropy
cross_entropy = tf.reduce_mean(
    categorical_crossentropy(labels, outputs))

In [29]:
# Add all the required variables to collections, so that they can 
# be easily retrieved while importing the meta_graph
tf.add_to_collection('cross_entropy', cross_entropy)
tf.add_to_collection('acc_value', acc_value)
tf.add_to_collection('inputs', inputs)
tf.add_to_collection('labels', labels)
tf.add_to_collection('predicted_class', predicted_class)

# We want to export only the common part of the graph i.e the 
# forward path and the loss value computation, so we export the 
# meta_graph and also initialize the saver here; this ensures that
# the unneeded parts of the graph are not exported.

# The meta_graph contains the information regarding the graph and
# the saver nodes. Note that by default, all of the collections 
# are exported and this is necessary for the retraining process.
meta_graph_proto = tf.train.export_meta_graph(filename = 'trained_model.meta')
# Initializing the Saver object adds nodes to save/restore the 
# parameters in the model which are currently defined. These 
# values can be loaded into the imported metagraph
saver = tf.train.Saver()

In [30]:
# Define the train_step; however this part of the graph does not
# get saved since the metagraph has already been exported.
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

In [31]:
# Create a dataset iterator to input the data to the model in batches
BATCH_SIZE = 128
num_epochs = 6
dataset = tf.data.Dataset.from_tensor_slices(
    (train_images, 
     train_labels)).batch(BATCH_SIZE).repeat(num_epochs)
iter = dataset.make_one_shot_iterator()
next_batch = iter.get_next()

In [3]:
# Train with the tf model with the correct dataset
with sess.as_default():
    init_var = tf.global_variables_initializer()
    init_var.run()
    # The training dataset gets repeatedly fed in, an exception 
    # indicates that training is done.
    while True:
        try:
            batch = sess.run([next_batch[0], next_batch[1]])
        except tf.errors.OutOfRangeError:
            print("Model trained for {} epochs".format(num_epochs))
            break
        train_step.run({inputs: batch[0], 
                        labels: batch[1]})
    # Measure test set accuracy after training
    print("accuracy on test set : {0:.3f}".format(acc_value.eval(
        feed_dict={inputs: test_images,
                   labels: test_labels})))
    # Get the original weight values for mse computation in 
    # the loss function
    weightVars = [Wconv1, Wconv2, Wconv3, Wdense, Wout]
    origWeights = [weightVar.eval() for weightVar in weightVars]
    biasVars = [biasConv1, biasConv2, biasConv3, biasDense, biasOut]
    origBiases = [biasVar.eval() for biasVar in biasVars]
# Tensorflow saves the model in 3 files, a meta file which contains
# the graph, a data file which is a binary file containing all the
# weight values, and an index file which helps tensorflow map the 
# contents of the data file to the actual tf variables.
# Since the meta file with the required graph has already been 
# saved, we need to reset the write_meta_graph flag so that the 
# graph saved earlier is not overwritten
save_path = saver.save(sess, "./trained_model", 
                       write_meta_graph=False)
print("Model saved in path: {}".format(save_path))

NameError: name 'sess' is not defined

In [ ]:
# Get the weight values from the correctly trained model 
# and store on disk so that they can be retrieved later
np.save('origWeights', origWeights)
np.save('origBiases', origBiases)

In [7]:
sess.close()